In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model, Sequential
from keras.layers import Input, Reshape, LSTM, Dense, Dropout
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../utils')
import KMeansDateRange
import get_open_close
import timeseries

"""K is the number of different
industries/sectors in the stock market"""
k = 11

In [2]:
df = get_open_close.populate_values_efficient("2012-05-18", "2020-04-01")

In [3]:
clusters = KMeansDateRange.kmeans_automated("2012-05-18", "2020-04-01", k=k)

In [6]:
%%time
mean_absolute_loss = 0

"""Randomly sampling 20 stocks from each cluster"""
for cluster in range(0, k):
    
    cluster_mae = 0
    sampled_stocks = clusters.loc[clusters['clusters'] == cluster].sample(n=20, random_state=1337)
    
    encoder_inputs = Input(shape=(14, 2))
    encoder_layer1 = LSTM(units=64, activation='relu', return_sequences=True, return_state=True)
    encoder_output1, _, _ = encoder_layer1(encoder_inputs)
    encoder_layer2 = LSTM(units=64, activation='relu', return_sequences=True, return_state=True)
    encoder_outputs, state_h, state_c = encoder_layer2(encoder_output1)
    encoder_states = [state_h, state_c]

    
    decoder_inputs = Input(shape=(1, 1))
    decoder_layer1 = LSTM(units=64, activation='relu', return_sequences=True, return_state=True)
    decoder_output1, _, _ = decoder_layer1(decoder_inputs, initial_state=encoder_states)
    decoder_layer2 = LSTM(units=64, activation='relu', return_sequences=False, return_state=True)
    decoder_outputs, _, _ = decoder_layer2(decoder_output1)

    decoder_dense = Dense(units=1, activation="linear")
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer="adam", loss="mae")
    
    stock_x_test = {}
    
    """Fit the data of each stock within a cluster on the RNN"""
    for index, stock in enumerate(sampled_stocks['symbols']):
        stock_data = df[stock]
        
        x_train, x_test = train_test_split(stock_data, test_size=0.3, shuffle=False)
        
        scaler = StandardScaler()
        x_train= scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        stock_x_test[stock] = x_test

        x_train, y_train = timeseries.get_timeseries_X_y(x_train)
        
        model.fit([x_train, y_train], y_train, validation_split=0.3, verbose=1,
                            initial_epoch=index*20, epochs=(index + 1)*20)
        
    for stock in stock_x_test:
        x_test, y_test = timeseries.get_timeseries_X_y(stock_x_test[stock])
        loss = model.evaluate([x_test, y_test], y_test)
        cluster_mae = cluster_mae + loss
    cluster_mae = cluster_mae/(sampled_stocks.shape[0])
    mean_absolute_loss = mean_absolute_loss + cluster_mae

print("Loss of Sequence-to-Sequence LSTM: ", mean_absolute_loss/k)

Epoch 1/20
30/30 [==============================] - 4s 34ms/step - loss: 0.6296 - val_loss: 0.3836
Epoch 2/20
30/30 [==============================] - 0s 14ms/step - loss: 0.2324 - val_loss: 0.0798
Epoch 3/20
30/30 [==============================] - 0s 15ms/step - loss: 0.1478 - val_loss: 0.0431
Epoch 4/20
30/30 [==============================] - 0s 14ms/step - loss: 0.1197 - val_loss: 0.0181
Epoch 5/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1078 - val_loss: 0.0207
Epoch 6/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1081 - val_loss: 0.0679
Epoch 7/20
30/30 [==============================] - 0s 14ms/step - loss: 0.0848 - val_loss: 0.0295
Epoch 8/20
30/30 [==============================] - 0s 13ms/step - loss: 0.0565 - val_loss: 0.0344
Epoch 9/20
30/30 [==============================] - 0s 14ms/step - loss: 0.0544 - val_loss: 0.0312
Epoch 10/20
30/30 [==============================] - 0s 13ms/step - loss: 0.0427 - val_loss: 0.0041
Epoch 11/

Epoch 83/100
30/30 [==============================] - 0s 12ms/step - loss: 0.0076 - val_loss: 0.0047
Epoch 84/100
30/30 [==============================] - 0s 14ms/step - loss: 0.0105 - val_loss: 0.0114
Epoch 85/100
30/30 [==============================] - 0s 13ms/step - loss: 0.0091 - val_loss: 0.0065
Epoch 86/100
30/30 [==============================] - 1s 17ms/step - loss: 0.0112 - val_loss: 0.0092
Epoch 87/100
30/30 [==============================] - 1s 22ms/step - loss: 0.0075 - val_loss: 0.0094
Epoch 88/100
30/30 [==============================] - 1s 18ms/step - loss: 0.0168 - val_loss: 0.0184
Epoch 89/100
30/30 [==============================] - 1s 20ms/step - loss: 0.0073 - val_loss: 0.0061
Epoch 90/100
30/30 [==============================] - 0s 15ms/step - loss: 0.0116 - val_loss: 0.0170
Epoch 91/100
30/30 [==============================] - 0s 16ms/step - loss: 0.0065 - val_loss: 0.0128
Epoch 92/100
30/30 [==============================] - 0s 14ms/step - loss: 0.0066 - val_los

30/30 [==============================] - 0s 12ms/step - loss: 0.0065 - val_loss: 0.0111
Epoch 164/180
30/30 [==============================] - 0s 12ms/step - loss: 0.0074 - val_loss: 0.0120
Epoch 165/180
30/30 [==============================] - 0s 12ms/step - loss: 0.0061 - val_loss: 0.0134
Epoch 166/180
30/30 [==============================] - 0s 12ms/step - loss: 0.0063 - val_loss: 0.0081
Epoch 167/180
30/30 [==============================] - 0s 12ms/step - loss: 0.0044 - val_loss: 0.0059
Epoch 168/180
30/30 [==============================] - 0s 14ms/step - loss: 0.0040 - val_loss: 0.0085
Epoch 169/180
30/30 [==============================] - 0s 13ms/step - loss: 0.0042 - val_loss: 0.0088
Epoch 170/180
30/30 [==============================] - 0s 12ms/step - loss: 0.0051 - val_loss: 0.0154
Epoch 171/180
30/30 [==============================] - 0s 12ms/step - loss: 0.0058 - val_loss: 0.0079
Epoch 172/180
30/30 [==============================] - 0s 13ms/step - loss: 0.0058 - val_loss: 0

30/30 [==============================] - 0s 12ms/step - loss: 0.0118 - val_loss: 0.0040
Epoch 244/260
30/30 [==============================] - 0s 12ms/step - loss: 0.0117 - val_loss: 0.0015
Epoch 245/260
30/30 [==============================] - 0s 12ms/step - loss: 0.0059 - val_loss: 0.0035
Epoch 246/260
30/30 [==============================] - 0s 12ms/step - loss: 0.0081 - val_loss: 0.0042
Epoch 247/260
30/30 [==============================] - 0s 16ms/step - loss: 0.0099 - val_loss: 0.0058
Epoch 248/260
30/30 [==============================] - 0s 14ms/step - loss: 0.0075 - val_loss: 0.0082
Epoch 249/260
30/30 [==============================] - 0s 14ms/step - loss: 0.0085 - val_loss: 0.0034
Epoch 250/260
30/30 [==============================] - 0s 16ms/step - loss: 0.0063 - val_loss: 0.0052
Epoch 251/260
30/30 [==============================] - 0s 14ms/step - loss: 0.0065 - val_loss: 0.0012
Epoch 252/260
30/30 [==============================] - 0s 14ms/step - loss: 0.0040 - val_loss: 0

30/30 [==============================] - 0s 12ms/step - loss: 0.0046 - val_loss: 0.0042
Epoch 324/340
30/30 [==============================] - 0s 12ms/step - loss: 0.0038 - val_loss: 0.0067
Epoch 325/340
30/30 [==============================] - 0s 13ms/step - loss: 0.0034 - val_loss: 0.0065
Epoch 326/340
30/30 [==============================] - 0s 12ms/step - loss: 0.0062 - val_loss: 0.0125
Epoch 327/340
30/30 [==============================] - 0s 15ms/step - loss: 0.0043 - val_loss: 0.0012
Epoch 328/340
30/30 [==============================] - 1s 18ms/step - loss: 0.0058 - val_loss: 0.0052
Epoch 329/340
30/30 [==============================] - 1s 20ms/step - loss: 0.0052 - val_loss: 0.0078
Epoch 330/340
30/30 [==============================] - 0s 15ms/step - loss: 0.0038 - val_loss: 8.1776e-04
Epoch 331/340
30/30 [==============================] - 0s 15ms/step - loss: 0.0029 - val_loss: 0.0124
Epoch 332/340
30/30 [==============================] - 0s 14ms/step - loss: 0.0034 - val_los

19/19 [==============================] - 0s 7ms/step - loss: 0.0076
Loss of Sequence-to-Sequence LSTM:  0.015872388426214457
CPU times: user 6min 26s, sys: 1min 17s, total: 7min 43s
Wall time: 2min 54s
